<a href="https://colab.research.google.com/github/Dinuda/wiki-stem-extraction/blob/main/extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install Wikipedia-API
import pandas as pd
import json
import wikipediaapi
import time
from requests.exceptions import ReadTimeout
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import cpu_count

In [4]:
import json

# Load the URLs from the JSON f|ile
with open("stem_url_bank-v2.json", "r") as f:
    url_bank = json.load(f)

# Display the number of URLs
print(f"Number of URLs in the bank: {len(url_bank)}")

try:
    with open("stem_url_bank-v2.json", "r") as f:
        url_bank = json.load(f)
except (IOError, json.JSONDecodeError) as e:
    print(f"Error loading JSON file: {e}")
    # Handle the error or exit gracefully
    url_bank = []

Number of URLs in the bank: 121683


In [7]:
def fetch_page(url):
  title = url.split("/")[-1]

  wiki = wikipediaapi.Wikipedia('STEM wiki (ydinuda@gmail.com)', 'en')

  page = wiki.page(title)

  if not page.exists():
    raise ValueError("Wikipedia page does not exists")

  def extract_sections(sections):
    result = []
    for s in sections:
        section_info = {
            'page_title': title,
            'section_title': s.title,
            'section_content': s.text
        }
        result.append(section_info)
        result.extend(extract_sections(s.sections))
    return result

  return extract_sections(page.sections)

In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [22]:
# compute advantage
len_url_bank = len(url_bank)

wiki_content = []
for i, url in enumerate(url_bank[:4]):
  url_content = fetch_page(url)
  if(url_content):
    print(f"{i} of {len_url_bank} extracted")
    wiki_content.append(url_content)
  else:
    print(f"{i} skipped")

# too bored some1 fix this
print(wiki_content[0])
df = pd.DataFrame(wiki_content[0])
output_file = "wiki_content.parquet"
pq.write_table(pa.Table.from_pandas(df), output_file)
print("Parquet file created successfully.")


0 skipped
1 of 121683 extracted
2 of 121683 extracted
3 of 121683 extracted
[{'page_title': 'FIPS_140', 'section_title': 'Purpose of FIPS 140', 'section_content': "The National Institute of Standards and Technology (NIST) issues the 140 Publication Series to coordinate the requirements and standards for cryptographic modules which include both hardware and software components for use by departments and agencies of the United States federal government. FIPS 140 does not purport to provide sufficient conditions to guarantee that a module conforming to its requirements is secure, still less that a system built using such modules is secure. The requirements cover not only the cryptographic modules themselves but also their documentation and (at the highest security level) some aspects of the comments contained in the source code.\nUser agencies desiring to implement cryptographic modules should confirm that the module they are using is covered by an existing validation certificate. FIPS 14

In [12]:
# Convert the data to a DataFrame
print(f"Shape of the combined DataFrame: {df.shape}")

Shape of the combined DataFrame: (4, 8)


In [24]:
df.head(3)


,page_title,section_title,section_content
0,FIPS_140,Purpose of FIPS 140,The National Institute of Standards and Techno...
1,FIPS_140,Security levels,"FIPS 140-2 defines four levels of security, si..."
2,FIPS_140,Scope of requirements,FIPS 140 imposes requirements in eleven differ...
